In [1]:
import numpy as np
from osgeo import gdal
gdal.UseExceptions()
import pygeoprocessing.geoprocessing as pgp
import os
import glob
import pandas as pd

In [2]:
workspace = '../data/bcdc_slr_regional_rec'
# base_uri = os.path.join(workspace, 'base_grid.tif')

In [3]:
nlcd_uri = '../data/bcdc_othernaturalareas/NaturalAreas_ForDave/nlcd_nodevt_utm.tif'
nlcd_src = gdal.Open(nlcd_uri)

In [65]:
def raster2array(raster_uri):
    '''
    helper function to go straight from raster filename to numpy array
    '''
    src = gdal.Open(raster_uri)
    band1 = src.GetRasterBand(1)
    rows = src.RasterYSize
    cols = src.RasterXSize
    arr = band1.ReadAsArray(0, 0, cols, rows)
    return(arr)

In [69]:
pud_uri = '../data/flickr/nlcd_grid_pud/pud_nlcdgrid.tif'
pud = raster2array(pud_uri)

In [ ]:
# pgp.new_raster_from_base(nlcd_uri, base_uri, gdal.GDT_Int16, [99], [0])

## Load AOIs and burn them onto a standard base raster

In [4]:
# BPAD
bpad_uri = os.path.join(workspace, 'bpad.tif')
pgp.new_raster_from_base(nlcd_uri, bpad_uri, gdal.GDT_Int16, [99], [0])
shp_uri = '../data/bcdc_othernaturalareas/NaturalAreas_ForDave/BayAreaProtectedLands_utm.shp'
pgp.rasterize(shp_uri, bpad_uri, [1], None)

In [5]:
# PCA
pca_uri = os.path.join(workspace, 'pca.tif')
pgp.new_raster_from_base(nlcd_uri, pca_uri, gdal.GDT_Int16, [99], [0])
shp_uri = '../data/pca/shapefiles/Priority_Conservation_Areas_current.shp'
pgp.rasterize(shp_uri, pca_uri, [1], None)

## then read AOIs as 2d arrays for querying

In [6]:
# Natural Lands - make 2d array 0s and 1s where 1s are all lulc classes other than development
# in the original datasource, development lulc's have already been assigned nodata
band1 = nlcd_src.GetRasterBand(1)
nodata = band1.GetNoDataValue()
rows = nlcd_src.RasterYSize
cols = nlcd_src.RasterXSize
vals = band1.ReadAsArray(0, 0, cols, rows)
nlcd = np.ones_like(vals)
nlcd[vals == nodata] = 0

In [66]:
bpad = raster2array(bpad_uri)
pca = raster2array(pca_uri)

In [71]:
nlcd.shape == bpad.shape == pca.shape == pud.shape

True

## Load SLR rasters, convert to 0s and 1s, align to base pixels, pad extents to match shape of base

In [57]:
def raster_to01(in_raster, out_raster):
    '''
    converts a raster of any datatype to Byte type with 0s replace nodata, 1s replacing all other data.
    '''
    print('reading 1')
    src = gdal.Open(in_raster)
    band1 = src.GetRasterBand(1)
    nodata = band1.GetNoDataValue()
    rows = src.RasterYSize
    cols = src.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)
    driver = src.GetDriver()
    
    print('assigning 0s and 1s')
    vals01 = np.ones_like(vals)
#     vals01[vals == nodata] = 0 # nodata was massive flt, this equality not happenin'
    vals01[vals <= 0] = 0
    
    out_data = driver.Create(out_raster, cols, rows, 1, gdal.GDT_Byte)
    out_band = out_data.GetRasterBand(1)

    print('writing 1')
    out_band.WriteArray(vals01)
    # flush data to disk, set the NoData value and calculate stats
    out_band.FlushCache()
    # out_band.SetNoDataValue(nodata)

    # georeference the image and set the projection
    out_data.SetGeoTransform(src.GetGeoTransform())
    out_data.SetProjection(src.GetProjection())
    del out_data

In [28]:
slr_root = '../data/pca/bcdc_slr/raster/'
slr_names = [os.path.basename(x) for x in glob.glob(os.path.join(slr_root,'*.tif'))]
slr_in_files = [os.path.join(slr_root, x) for x in slr_names]
slr_out_files = [os.path.join(slr_root, 'zeros_and_ones', x) for x in slr_names]

In [58]:
[raster_to01(f1, f2) for f1, f2 in zip(slr_in_files, slr_out_files)]

reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1
reading 1
assigning 0s and 1s
writing 1


[None, None, None, None, None, None, None, None, None, None]

In [59]:
align_rasters = [nlcd_uri] + [os.path.join(slr_root, 'zeros_and_ones', x) for x in slr_names]
target_rasters = [os.path.join(slr_root, 'aligned', 'nlcd_utm.tif')] + [os.path.join(slr_root, 'aligned', x) for x in slr_names]

In [26]:
pixel_size = pgp.get_raster_info(nlcd_uri)['pixel_size']
bbox = pgp.get_raster_info(nlcd_uri)['bounding_box']

In [62]:
pgp.align_and_resize_raster_stack(align_rasters, target_rasters, \
                                  ["nearest"]*len(align_rasters), pixel_size, bbox, \
                                  raster_align_index=0)

### testing some logic on small arrays...

In [95]:
x = np.zeros((3,3))
x[:2,:2] = 1
y = np.zeros((3,3))
y[:1,:2] = 1
w = np.zeros((3,3))
w[0,1] = 1
z = np.flip(np.arange(9).reshape(3, 3), 0)

In [96]:
x

array([[1., 1., 0.],
       [1., 1., 0.],
       [0., 0., 0.]])

In [109]:
y

array([[1., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [114]:
w

array([[0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [104]:
z

array([[6, 7, 8],
       [3, 4, 5],
       [0, 1, 2]])

In [177]:
z[(x != 0) & (y != 0) & (w != 1)]

array([6])

## All calculations below exclude developed areas (based on lulc development classes)

In [118]:
slr_rasters = glob.glob(os.path.join(slr_root, 'aligned','Inundate*.tif'))

In [181]:
# In PCA network
pca_slr = {}

for slr_uri in slr_rasters:
    print(slr_uri)
    slr = raster2array(slr_uri)
    pud.shape == nlcd.shape == pca.shape == slr.shape
    puds = np.sum(pud[(nlcd != 0) & \
                      (pca != 0) & \
                      (slr != 0)])
    pca_slr[os.path.basename(slr_uri)] = puds
    
puds_slr0 = np.sum(pud[(nlcd != 0) & \
                      (pca != 0)])
pca_slr['Inundate_0'] = puds_slr0

../data/pca/bcdc_slr/raster/aligned/Inundate_66.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_77.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_12.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_52.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_108.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_96.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_48.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_24.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_36.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_84.tif


In [178]:
# In BPAD network and not in PCA network
bpad_slr = {}

for slr_uri in slr_rasters:
    print(slr_uri)
    slr = raster2array(slr_uri)
    puds = np.sum(pud[(nlcd != 0) & \
                      (bpad != 0) & \
                      (slr != 0) & \
                      (pca != 1)])
    bpad_slr[os.path.basename(slr_uri)] = puds
        
puds_slr0 = np.sum(pud[(nlcd != 0) & \
                      (bpad != 0) & \
                      (pca != 1)])
bpad_slr['Inundate_0'] = puds_slr0

../data/pca/bcdc_slr/raster/aligned/Inundate_66.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_77.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_12.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_52.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_108.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_96.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_48.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_24.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_36.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_84.tif


In [173]:
# In Natural Lands and not in BPAD and PCA networks
nlcd_slr = {}

for slr_uri in slr_rasters:
    print(slr_uri)
    slr = raster2array(slr_uri)
    puds = np.sum(pud[(nlcd != 0) & \
                      (slr != 0) & \
                      (pca != 1) & \
                      (bpad != 1)])
    
    nlcd_slr[os.path.basename(slr_uri)] = puds
    
puds_slr0 = np.sum(pud[(nlcd != 0) & \
                      (pca != 1) & \
                      (bpad != 1)])
nlcd_slr['Inundate_0'] = puds_slr0

../data/pca/bcdc_slr/raster/aligned/Inundate_66.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_77.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_12.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_52.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_108.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_96.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_48.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_24.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_36.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_84.tif


## Compile results

In [186]:
pca_df = pd.DataFrame(pca_slr.items(), columns=['scenario', 'pca'])
bpad_df = pd.DataFrame(bpad_slr.items(), columns=['scenario', 'bpad'])
nlcd_df = pd.DataFrame(nlcd_slr.items(), columns=['scenario', 'nlcd'])

In [187]:
results = pd.merge(pd.merge(pca_df, bpad_df, on='scenario'), nlcd_df, on='scenario')

In [204]:
results['slr_inches'] = pd.to_numeric(results['scenario'].str.extract('([0-9]?[0-9]?[0-9])'))

/home/dmf/miniconda3/envs/geowork2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [205]:
results.sort_values(by='slr_inches')

,scenario,pca,bpad,nlcd,slr_inches
2,Inundate_0,147464,194652,125562,0
10,Inundate_12.tif,1982,3356,2824,12
7,Inundate_24.tif,4042,4698,4056,24
0,Inundate_36.tif,4590,7014,5450,36
1,Inundate_48.tif,5428,8546,6818,48
9,Inundate_52.tif,5540,9050,7570,52
4,Inundate_66.tif,5992,12068,8532,66
5,Inundate_77.tif,7640,12716,9712,77
6,Inundate_84.tif,7786,13310,10102,84
3,Inundate_96.tif,7916,14320,10884,96


In [206]:
results.to_csv(os.path.join(workspace, 'results.csv'))